In [1]:
import numpy as np
import os
import glob
import math
import nibabel as nib
import cv2

from pycimg import CImg
from skimage import measure
from skimage import morphology
from scipy import ndimage
from scipy.spatial import ConvexHull


In [2]:
fname = './Images/IMG_0001.nii.gz'

foo = nib.load(fname).get_fdata()

print(foo.shape)
#CImg(foo).display();


(512, 512, 133)


In [3]:
from scipy import ndimage as ndi
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)

denoised = ndi.median_filter(foo, size=3)

In [4]:
CImg(denoised).display();

In [5]:
li_thresholded = denoised > filters.threshold_li(denoised) * 0.3

filled = ndi.binary_fill_holes(li_thresholded)

In [6]:
CImg(filled).display();

In [7]:
width = 45

processed = morphology.remove_small_holes(
    filled, 
    area_threshold=width ** 3
)

processed = morphology.remove_small_objects(
    processed, 
    min_size=width ** 3
)

In [8]:
import numpy as np
from scipy import ndimage
from skimage.measure import label, regionprops

labeled_image, num_features = ndimage.label(processed)

segment_sizes = [r.area for r in regionprops(labeled_image)]

sorted_segment_sizes = sorted(enumerate(segment_sizes, 1), key=lambda x: x[1], reverse=True)

mask = np.zeros_like(labeled_image, dtype=bool)

largest_label, _ = sorted_segment_sizes[0]
mask[labeled_image == largest_label] = True

lung_data = np.where(mask, labeled_image, 0)

In [10]:
CImg(lung_data).display();

In [17]:
lung_reverse = 1 - lung_data

In [19]:
CImg(lung_reverse).display();

In [20]:
labeled_image, num_features = ndimage.label(lung_reverse)

segment_sizes = np.bincount(labeled_image.flat)

largest_segment_label = np.argmax(segment_sizes[1:]) + 1

largest_segment_mask = (labeled_image == largest_segment_label)

largest_segment_image = largest_segment_mask.astype(int)

In [22]:
final_image = 1 - largest_segment_image

In [24]:
CImg(final_image).display();